In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [3]:
TRAINING_SIZE = 30000
DIGITS = 3

REVERSE = True

MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1


In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)


In [5]:
ctable = CharacterTable(chars)


# Data Generation

In [6]:
questions = []
expected = []
seen = set()

print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                            for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    # if operator == 1 do adder, if operator == 2 do substractor
    operator = np.random.choice(list('12'))
    if operator == '1' :
        
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        
    elif operator == '2' :
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a - b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))

    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))



Generating data...
Total addition questions: 30000


In [7]:
print(questions[:10], '\n' ,expected[:10])


['17+5   ', '97-84  ', '880+188', '507-105', '26+88  ', '4+98   ', '6+4    ', '41+27  ', '2-114  ', '482+30 '] 
 ['22  ', '13  ', '1068', '402 ', '114 ', '102 ', '10  ', '68  ', '-112', '512 ']


# Processing

In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [9]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:25000]
train_y = y[:25000]
test_x = x[25000:]
test_y = y[25000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)



Training Data:
(22500, 7, 13)
(22500, 4, 13)
Validation Data:
(2500, 7, 13)
(2500, 4, 13)
Testing Data:
(5000, 7, 13)
(5000, 4, 13)


In [10]:
#print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])


# Build Model

In [11]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()

# "Encode" the input sequence using an RNN, input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# the decoder RNN's input
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN  with a single layer.
for _ in range(LAYERS):
    # all the outputs in the form of (num_samples, timesteps, output_dim). 
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))


model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

############################################
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training, change epoch here

In [12]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=2,
              validation_data=(x_val, y_val))
    # visualizing validation set prediction errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 32s 1ms/step - loss: 2.0801 - acc: 0.2850 - val_loss: 1.8712 - val_acc: 0.3362
Epoch 2/2
22500/22500 [==============================] - 10s 450us/step - loss: 1.7991 - acc: 0.3499 - val_loss: 1.7672 - val_acc: 0.3571
Q  175+18 T 652  ☒ 104 
Q   0-462 T 264  ☒ 22  
Q   78+59 T 182  ☒ 104 
Q  073-85 T -312 ☒ -119
Q  21-375 T 561  ☒ 32  
Q 999-872 T -721 ☒ -111
Q   83-62 T -12  ☒ -2  
Q 911+576 T 794  ☒ 101 
Q  43+088 T 914  ☒ 104 
Q   756-0 T -657 ☒ -779

--------------------------------------------------
Iteration 1
Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 10s 446us/step - loss: 1.7413 - acc: 0.3628 - val_loss: 1.7286 - val_acc: 0.3645
Epoch 2/2
22500/22500 [==============================] - 10s 428us/step - loss: 1.7031 - acc: 0.3728 - val_loss: 1.6924 - 

22500/22500 [==============================] - 13s 556us/step - loss: 1.0407 - acc: 0.6175 - val_loss: 1.0759 - val_acc: 0.5987
Q 113+238 T 1143 ☒ 1172
Q  56-783 T 322  ☒ 311 
Q 928-026 T -209 ☒ -211
Q   22-25 T 30   ☒ 25  
Q    29-9 T -83  ☒ -73 
Q  612+01 T 226  ☒ 138 
Q 412-163 T 147  ☒ 114 
Q   3+533 T 338  ☒ 348 
Q   526-2 T -623 ☒ -624
Q 055+718 T 1367 ☒ 1363

--------------------------------------------------
Iteration 12
Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 12s 530us/step - loss: 1.0263 - acc: 0.6228 - val_loss: 1.0666 - val_acc: 0.6009
Epoch 2/2
22500/22500 [==============================] - 12s 530us/step - loss: 1.0055 - acc: 0.6312 - val_loss: 1.0523 - val_acc: 0.6089
Q   01+73 T 47   ☒ 45  
Q  565+86 T 633  ☒ 661 
Q   5-747 T 742  ☒ 747 
Q   481+9 T 193  ☒ 196 
Q   18-35 T -28  ☒ -24 
Q 296-809 T 216  ☒ 144 
Q  072-55 T -215 ☒ -227
Q   052+5 T 255  ☒ 251 
Q   5-113 T 306  ☒ 316 
Q   5+392 T 298  ☒ 281 

-

22500/22500 [==============================] - 12s 527us/step - loss: 0.6832 - acc: 0.7529 - val_loss: 0.8608 - val_acc: 0.6775
Epoch 2/2
22500/22500 [==============================] - 12s 529us/step - loss: 0.6710 - acc: 0.7571 - val_loss: 0.8399 - val_acc: 0.6835
Q 481+759 T 1141 ☒ 1043
Q   52-68 T 61   ☒ 69  
Q  012+13 T 241  ☒ 235 
Q   16+32 T 84   ☒ 77  
Q 565+643 T 911  ☑ 911 
Q 576+521 T 800  ☒ 890 
Q  62-484 T 458  ☒ 463 
Q  67+212 T 288  ☒ 286 
Q  353-97 T -274 ☒ -276
Q  53+559 T 990  ☑ 990 

--------------------------------------------------
Iteration 26
Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 12s 517us/step - loss: 0.6587 - acc: 0.7620 - val_loss: 0.8308 - val_acc: 0.6830
Epoch 2/2
22500/22500 [==============================] - 12s 550us/step - loss: 0.6476 - acc: 0.7648 - val_loss: 0.8161 - val_acc: 0.6899
Q   62+41 T 40   ☒ 37  
Q  81-289 T 964  ☒ 960 
Q  463+54 T 409  ☒ 417 
Q  599+87 T 1073 ☒ 1077
Q   0-65

22500/22500 [==============================] - 9s 412us/step - loss: 0.2264 - acc: 0.9355 - val_loss: 0.5283 - val_acc: 0.8134
Epoch 2/2
22500/22500 [==============================] - 9s 409us/step - loss: 0.2241 - acc: 0.9344 - val_loss: 0.4833 - val_acc: 0.8173
Q   16+32 T 84   ☑ 84  
Q    76-4 T -63  ☑ -63 
Q  791+94 T 246  ☒ 236 
Q   3+595 T 598  ☑ 598 
Q 244+957 T 1201 ☒ 1291
Q  392+99 T 392  ☒ 382 
Q 859+823 T 1286 ☒ 1287
Q  188-36 T -818 ☒ -829
Q   839-2 T -936 ☑ -936
Q  903+59 T 404  ☒ 325 

--------------------------------------------------
Iteration 44
Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 9s 416us/step - loss: 0.2100 - acc: 0.9417 - val_loss: 0.4803 - val_acc: 0.8254
Epoch 2/2
22500/22500 [==============================] - 9s 417us/step - loss: 0.1974 - acc: 0.9464 - val_loss: 0.4731 - val_acc: 0.8253
Q   4-774 T 473  ☑ 473 
Q   13-73 T 6    ☒ 74  
Q  76+975 T 646  ☑ 646 
Q   7-612 T 209  ☒ 219 
Q 673+402 T 

22500/22500 [==============================] - 6s 277us/step - loss: 0.0134 - acc: 0.9997 - val_loss: 0.4472 - val_acc: 0.8772
Epoch 2/2
22500/22500 [==============================] - 6s 280us/step - loss: 0.0445 - acc: 0.9905 - val_loss: 0.4510 - val_acc: 0.8736
Q   342-2 T -241 ☑ -241
Q 563+947 T 1114 ☒ 1115
Q   46-66 T 2    ☒ 1   
Q 947+718 T 1566 ☑ 1566
Q   217-1 T -711 ☒ -712
Q  97+023 T 399  ☑ 399 
Q  28-582 T 203  ☒ 103 
Q   9+985 T 598  ☑ 598 
Q   14-43 T -7   ☑ -7  
Q   84+24 T 90   ☑ 90  

--------------------------------------------------
Iteration 84
Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 6s 272us/step - loss: 0.0214 - acc: 0.9986 - val_loss: 0.4194 - val_acc: 0.8834
Epoch 2/2
22500/22500 [==============================] - 6s 284us/step - loss: 0.0151 - acc: 0.9998 - val_loss: 0.4184 - val_acc: 0.8826
Q   087-0 T -780 ☑ -780
Q    1+98 T 90   ☑ 90  
Q 381+425 T 707  ☑ 707 
Q 291+467 T 956  ☑ 956 
Q 237-771 T 

22500/22500 [==============================] - 7s 290us/step - loss: 0.0079 - acc: 1.0000 - val_loss: 0.4356 - val_acc: 0.8853
Epoch 2/2
22500/22500 [==============================] - 7s 291us/step - loss: 0.0076 - acc: 1.0000 - val_loss: 0.4387 - val_acc: 0.8821
Q  98+905 T 598  ☑ 598 
Q  54+993 T 444  ☑ 444 
Q 097+203 T 1092 ☑ 1092
Q 027-583 T -335 ☒ -346
Q  417+53 T 749  ☑ 749 
Q  049+32 T 963  ☒ 954 
Q  463+54 T 409  ☑ 409 
Q 352-033 T 77   ☒ 7   
Q 071-721 T -43  ☒ -36 
Q   61+88 T 104  ☑ 104 

--------------------------------------------------
Iteration 96
Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 6s 285us/step - loss: 0.0073 - acc: 1.0000 - val_loss: 0.4412 - val_acc: 0.8838
Epoch 2/2
22500/22500 [==============================] - 7s 298us/step - loss: 0.0071 - acc: 1.0000 - val_loss: 0.4457 - val_acc: 0.8852
Q   4-315 T 509  ☒ 510 
Q  398+89 T 991  ☑ 991 
Q   546-2 T -643 ☑ -643
Q  315-89 T -415 ☒ -416
Q  223+06 T 

# Testing

In [13]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 

preds = model.predict_classes(test_x, verbose=0)

i = 0
count = 0
while i < len(preds) :
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)

    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    i += 1
    
testing_acc = (count/len(test_y))*100
print('-'*60)
print('-'*30, colors.ok + str(testing_acc) + "%" + colors.close, '-'*30)
print('-'*60)
#####################################################
    

MSG : Prediction
Q 420+2   T 422  ☑ 422 
Q 44-31   T 13   ☒ 12  
Q 484+27  T 511  ☒ 501 
Q 8-955   T -947 ☑ -947
Q 77+36   T 113  ☑ 113 
Q 322+19  T 341  ☑ 341 
Q 2+970   T 972  ☑ 972 
Q 767+65  T 832  ☑ 832 
Q 528+871 T 1399 ☑ 1399
Q 649+33  T 682  ☑ 682 
Q 315-471 T -156 ☒ -266
Q 594+8   T 602  ☒ 502 
Q 160-36  T 124  ☑ 124 
Q 22+58   T 80   ☒ 70  
Q 541+94  T 635  ☑ 635 
Q 58+314  T 372  ☑ 372 
Q 9+390   T 399  ☑ 399 
Q 577-29  T 548  ☑ 548 
Q 59+14   T 73   ☒ 83  
Q 431+1   T 432  ☑ 432 
Q 684+54  T 738  ☑ 738 
Q 89-322  T -233 ☒ -223
Q 886+93  T 979  ☑ 979 
Q 5+182   T 187  ☑ 187 
Q 28-58   T -30  ☑ -30 
Q 58-2    T 56   ☑ 56  
Q 2+182   T 184  ☑ 184 
Q 459+793 T 1252 ☒ 1251
Q 661+52  T 713  ☑ 713 
Q 9+782   T 791  ☑ 791 
Q 825-983 T -158 ☒ -168
Q 509+86  T 595  ☑ 595 
Q 632+57  T 689  ☒ 699 
Q 949+771 T 1720 ☑ 1720
Q 537-84  T 453  ☒ 454 
Q 4+680   T 684  ☑ 684 
Q 16+538  T 554  ☑ 554 
Q 50-662  T -612 ☑ -612
Q 48-200  T -152 ☑ -152
Q 79-44   T 35   ☑ 35  
Q 54+280  T 334  ☑ 334 

Q 47+16   T 63   ☑ 63  
Q 345+39  T 384  ☒ 394 
Q 909-212 T 697  ☒ 787 
Q 8-296   T -288 ☑ -288
Q 97-323  T -226 ☒ -225
Q 5-23    T -18  ☑ -18 
Q 581+66  T 647  ☑ 647 
Q 411-7   T 404  ☑ 404 
Q 634-63  T 571  ☑ 571 
Q 43+2    T 45   ☑ 45  
Q 744+84  T 828  ☑ 828 
Q 3-380   T -377 ☑ -377
Q 283+743 T 1026 ☒ 1106
Q 810-58  T 752  ☒ 742 
Q 18-4    T 14   ☑ 14  
Q 523-8   T 515  ☑ 515 
Q 850-23  T 827  ☑ 827 
Q 6-783   T -777 ☑ -777
Q 4-983   T -979 ☑ -979
Q 8+499   T 507  ☑ 507 
Q 34-56   T -22  ☑ -22 
Q 575+51  T 626  ☑ 626 
Q 74+98   T 172  ☑ 172 
Q 461+962 T 1423 ☒ 1412
Q 66-65   T 1    ☑ 1   
Q 722+333 T 1055 ☒ 1054
Q 850-86  T 764  ☑ 764 
Q 735+72  T 807  ☑ 807 
Q 19-250  T -231 ☑ -231
Q 10-849  T -839 ☑ -839
Q 0-769   T -769 ☒ -779
Q 263+39  T 302  ☑ 302 
Q 268+0   T 268  ☑ 268 
Q 959+64  T 1023 ☑ 1023
Q 199+52  T 251  ☒ 241 
Q 478+43  T 521  ☑ 521 
Q 264-50  T 214  ☒ 104 
Q 771+862 T 1633 ☒ 1623
Q 0+641   T 641  ☑ 641 
Q 41+26   T 67   ☑ 67  
Q 9-290   T -281 ☑ -281
Q 440-62  T 378 

Q 88-415  T -327 ☑ -327
Q 6-837   T -831 ☑ -831
Q 11-555  T -544 ☑ -544
Q 862-15  T 847  ☑ 847 
Q 5-507   T -502 ☑ -502
Q 825-43  T 782  ☑ 782 
Q 66+135  T 201  ☑ 201 
Q 878-82  T 796  ☒ 896 
Q 0-627   T -627 ☑ -627
Q 10-64   T -54  ☑ -54 
Q 94-354  T -260 ☑ -260
Q 92+447  T 539  ☒ 530 
Q 425+41  T 466  ☑ 466 
Q 359-11  T 348  ☒ 358 
Q 709-563 T 146  ☒ 145 
Q 349-8   T 341  ☑ 341 
Q 51+509  T 560  ☑ 560 
Q 991-0   T 991  ☑ 991 
Q 159+88  T 247  ☑ 247 
Q 446+551 T 997  ☒ 909 
Q 768+94  T 862  ☑ 862 
Q 482-225 T 257  ☒ 267 
Q 16+716  T 732  ☑ 732 
Q 81+11   T 92   ☑ 92  
Q 897-8   T 889  ☒ 899 
Q 1-280   T -279 ☒ -289
Q 36+782  T 818  ☑ 818 
Q 553+50  T 603  ☒ 503 
Q 634+51  T 685  ☑ 685 
Q 876+2   T 878  ☑ 878 
Q 293-7   T 286  ☑ 286 
Q 72+84   T 156  ☑ 156 
Q 660-85  T 575  ☑ 575 
Q 227-974 T -747 ☒ -744
Q 6-1     T 5    ☑ 5   
Q 88-894  T -806 ☑ -806
Q 42+254  T 296  ☑ 296 
Q 2+291   T 293  ☒ 294 
Q 219+23  T 242  ☑ 242 
Q 5-271   T -266 ☑ -266
Q 84-436  T -352 ☑ -352
Q 838-57  T 781 

Q 576+32  T 608  ☒ 508 
Q 155-89  T 66   ☒ 56  
Q 58+208  T 266  ☑ 266 
Q 84+160  T 244  ☑ 244 
Q 4+70    T 74   ☑ 74  
Q 47-754  T -707 ☑ -707
Q 568+479 T 1047 ☒ 1057
Q 209-850 T -641 ☒ -541
Q 203-315 T -112 ☒ -111
Q 759+75  T 834  ☑ 834 
Q 410-8   T 402  ☑ 402 
Q 103+876 T 979  ☑ 979 
Q 359+2   T 361  ☑ 361 
Q 369-30  T 339  ☒ 349 
Q 517-391 T 126  ☒ 176 
Q 601+965 T 1566 ☒ 1576
Q 12+476  T 488  ☑ 488 
Q 4+289   T 293  ☑ 293 
Q 117-2   T 115  ☑ 115 
Q 281-272 T 9    ☒ 98  
Q 9-644   T -635 ☒ -634
Q 77-29   T 48   ☑ 48  
Q 264-61  T 203  ☒ 193 
Q 57-219  T -162 ☑ -162
Q 1-250   T -249 ☑ -249
Q 817+4   T 821  ☑ 821 
Q 31+796  T 827  ☒ 837 
Q 70-970  T -900 ☒ -901
Q 328+72  T 400  ☑ 400 
Q 24-47   T -23  ☑ -23 
Q 31-840  T -809 ☑ -809
Q 625-35  T 590  ☒ 580 
Q 28-841  T -813 ☑ -813
Q 538-4   T 534  ☑ 534 
Q 4+615   T 619  ☑ 619 
Q 176+159 T 335  ☒ 334 
Q 887-982 T -95  ☒ -18 
Q 689+92  T 781  ☑ 781 
Q 82-54   T 28   ☒ 38  
Q 12-44   T -32  ☑ -32 
Q 1+694   T 695  ☑ 695 
Q 49-346  T -297

Q 18+87   T 105  ☑ 105 
Q 74+78   T 152  ☑ 152 
Q 289-79  T 210  ☑ 210 
Q 500+56  T 556  ☑ 556 
Q 90+323  T 413  ☒ 412 
Q 698+41  T 739  ☒ 749 
Q 634-38  T 596  ☒ 696 
Q 725-629 T 96   ☒ 115 
Q 130+380 T 510  ☒ 431 
Q 12+71   T 83   ☑ 83  
Q 97+27   T 124  ☑ 124 
Q 7+738   T 745  ☑ 745 
Q 973-4   T 969  ☒ 970 
Q 83-571  T -488 ☒ -487
Q 529-48  T 481  ☑ 481 
Q 546-70  T 476  ☒ 477 
Q 506-949 T -443 ☒ -442
Q 308-565 T -257 ☒ -264
Q 13+294  T 307  ☑ 307 
Q 6-2     T 4    ☒ 3   
Q 80+5    T 85   ☒ 84  
Q 66-243  T -177 ☑ -177
Q 889-61  T 828  ☑ 828 
Q 37-367  T -330 ☒ -331
Q 7-509   T -502 ☑ -502
Q 62-83   T -21  ☒ -22 
Q 109+7   T 116  ☒ 115 
Q 5+865   T 870  ☒ 860 
Q 463-67  T 396  ☒ 386 
Q 161+139 T 300  ☒ 380 
Q 419-26  T 393  ☑ 393 
Q 830-546 T 284  ☒ 276 
Q 21+10   T 31   ☑ 31  
Q 66+30   T 96   ☑ 96  
Q 354+505 T 859  ☒ 850 
Q 133-28  T 105  ☒ 106 
Q 28+819  T 847  ☒ 857 
Q 258+400 T 658  ☒ 668 
Q 322-472 T -150 ☒ -120
Q 106+82  T 188  ☑ 188 
Q 2-595   T -593 ☑ -593
Q 901-3   T 898 

...

Q 678+496 T 1174 ☒ 1184
Q 905+115 T 1020 ☒ 1010
Q 63-426  T -363 ☑ -363
Q 909+897 T 1806 ☒ 1809
Q 70-57   T 13   ☑ 13  
Q 146-6   T 140  ☒ 131 
Q 44-46   T -2   ☒ -1  
Q 894+30  T 924  ☑ 924 
Q 53+43   T 96   ☑ 96  
Q 52+482  T 534  ☑ 534 
Q 741+45  T 786  ☑ 786 
Q 415-98  T 317  ☒ 327 
Q 43+893  T 936  ☑ 936 
Q 318-660 T -342 ☒ -341
Q 349-465 T -116 ☑ -116
Q 836+792 T 1628 ☒ 1629
Q 5+992   T 997  ☑ 997 
Q 658-17  T 641  ☒ 631 
Q 8-760   T -752 ☑ -752
Q 3-300   T -297 ☑ -297
Q 541+782 T 1323 ☒ 1312
Q 120+913 T 1033 ☒ 1023
Q 7+582   T 589  ☑ 589 
Q 83+497  T 580  ☒ 570 
Q 2+881   T 883  ☑ 883 
Q 169-60  T 109  ☒ 100 
Q 92-37   T 55   ☒ 56  
Q 65+920  T 985  ☑ 985 
Q 72+671  T 743  ☑ 743 
Q 12+941  T 953  ☒ 954 
Q 544+87  T 631  ☑ 631 
Q 366-38  T 328  ☑ 328 
Q 72-596  T -524 ☑ -524
Q 774-82  T 692  ☒ 693 
Q 401-534 T -133 ☒ -244
Q 4-681   T -677 ☑ -677
Q 96-4    T 92   ☒ 91  
Q 192+57  T 249  ☑ 249 
Q 420-81  T 339  ☒ 340 
Q 94-53   T 41   ☑ 41  
Q 4-611   T -607 ☒ -606
Q 49-74   T -25 

Q 488+27  T 515  ☑ 515 
Q 238+773 T 1011 ☑ 1011
Q 4-527   T -523 ☑ -523
Q 458+88  T 546  ☑ 546 
Q 61-814  T -753 ☑ -753
Q 425+481 T 906  ☑ 906 
Q 69+230  T 299  ☒ 399 
Q 27+871  T 898  ☒ 908 
Q 749-768 T -19  ☒ -89 
Q 43+68   T 111  ☑ 111 
Q 285-443 T -158 ☒ -150
Q 570+2   T 572  ☑ 572 
Q 115+68  T 183  ☑ 183 
Q 72+54   T 126  ☑ 126 
Q 856-934 T -78  ☒ -7  
Q 77+580  T 657  ☑ 657 
Q 879+3   T 882  ☑ 882 
Q 8+411   T 419  ☒ 429 
Q 11-22   T -11  ☒ -10 
Q 542-53  T 489  ☒ 499 
Q 936+7   T 943  ☑ 943 
Q 27+44   T 71   ☑ 71  
Q 61+381  T 442  ☑ 442 
Q 457-9   T 448  ☑ 448 
Q 60+120  T 180  ☑ 180 
Q 1+179   T 180  ☑ 180 
Q 94-684  T -590 ☒ -690
Q 259-72  T 187  ☒ 987 
Q 65+777  T 842  ☑ 842 
Q 491+9   T 500  ☒ 490 
Q 35-844  T -809 ☒ -800
Q 24-1    T 23   ☑ 23  
Q 568-7   T 561  ☑ 561 
Q 393-14  T 379  ☒ 370 
Q 778-18  T 760  ☑ 760 
Q 4+820   T 824  ☑ 824 
Q 5+200   T 205  ☒ 204 
Q 842+249 T 1091 ☒ 1092
Q 0+810   T 810  ☒ 800 
Q 898+554 T 1452 ☑ 1452
Q 39-90   T -51  ☒ -50 
Q 527-870 T -343

Q 326+48  T 374  ☑ 374 
Q 1+50    T 51   ☑ 51  
Q 215+30  T 245  ☒ 246 
Q 82-368  T -286 ☒ -285
Q 71-72   T -1   ☒ -   
Q 930+699 T 1629 ☑ 1629
Q 11-971  T -960 ☒ -961
Q 143-76  T 67   ☒ 66  
Q 648+43  T 691  ☑ 691 
Q 33+887  T 920  ☑ 920 
Q 57+323  T 380  ☒ 370 
Q 694+84  T 778  ☑ 778 
Q 26-617  T -591 ☒ -691
Q 574+445 T 1019 ☑ 1019
Q 34+403  T 437  ☑ 437 
Q 50-862  T -812 ☒ -811
Q 46-29   T 17   ☒ 1   
Q 98-784  T -686 ☑ -686
Q 882+3   T 885  ☑ 885 
Q 735+494 T 1229 ☒ 1319
Q 374-46  T 328  ☑ 328 
Q 51+15   T 66   ☑ 66  
Q 966-223 T 743  ☒ 623 
Q 0+169   T 169  ☑ 169 
Q 575+334 T 909  ☒ 999 
Q 24+334  T 358  ☑ 358 
Q 784-9   T 775  ☑ 775 
Q 393-36  T 357  ☒ 356 
Q 316-7   T 309  ☑ 309 
Q 9-59    T -50  ☑ -50 
Q 3-629   T -626 ☒ -624
Q 14-474  T -460 ☑ -460
Q 983-2   T 981  ☑ 981 
Q 1+622   T 623  ☑ 623 
Q 340-74  T 266  ☒ 267 
Q 13-79   T -66  ☑ -66 
Q 583+13  T 596  ☑ 596 
Q 540-94  T 446  ☑ 446 
Q 258+20  T 278  ☑ 278 
Q 35+72   T 107  ☒ 117 
Q 76+81   T 157  ☑ 157 
Q 243-105 T 138 

Q 874+4   T 878  ☑ 878 
Q 62+94   T 156  ☑ 156 
Q 544+1   T 545  ☑ 545 
Q 71+299  T 370  ☑ 370 
Q 28+632  T 660  ☒ 650 
Q 767+9   T 776  ☑ 776 
Q 159-847 T -688 ☒ -698
Q 26+658  T 684  ☑ 684 
Q 75-391  T -316 ☑ -316
Q 43+237  T 280  ☑ 280 
Q 1+269   T 270  ☑ 270 
Q 75+870  T 945  ☑ 945 
Q 144+562 T 706  ☑ 706 
Q 895-708 T 187  ☑ 187 
Q 51+751  T 802  ☑ 802 
Q 176+702 T 878  ☑ 878 
Q 3+867   T 870  ☑ 870 
Q 720-5   T 715  ☑ 715 
Q 866+276 T 1142 ☒ 1243
Q 581-298 T 283  ☒ 282 
Q 24+554  T 578  ☑ 578 
Q 903-66  T 837  ☒ 847 
Q 681-54  T 627  ☒ 626 
Q 96-31   T 65   ☑ 65  
Q 457+18  T 475  ☑ 475 
Q 660-861 T -201 ☒ -291
Q 848+4   T 852  ☑ 852 
Q 205-912 T -707 ☒ -717
Q 2+120   T 122  ☑ 122 
Q 775-6   T 769  ☑ 769 
Q 7-702   T -695 ☑ -695
Q 747-40  T 707  ☑ 707 
Q 89+901  T 990  ☑ 990 
Q 391+0   T 391  ☑ 391 
Q 9-579   T -570 ☑ -570
Q 477+8   T 485  ☑ 485 
Q 387+47  T 434  ☑ 434 
Q 358-2   T 356  ☑ 356 
Q 752-32  T 720  ☒ 711 
Q 70-36   T 34   ☑ 34  
Q 494+40  T 534  ☑ 534 
Q 893-23  T 870 

Q 900-8   T 892  ☒ 893 
Q 0-15    T -15  ☒ -14 
Q 4-764   T -760 ☑ -760
Q 797-60  T 737  ☑ 737 
Q 892-83  T 809  ☑ 809 
Q 94-98   T -4   ☒ -5  
Q 167+410 T 577  ☒ 687 
Q 8+747   T 755  ☑ 755 
Q 266-5   T 261  ☑ 261 
Q 81+524  T 605  ☒ 505 
Q 501+9   T 510  ☒ 500 
Q 421+5   T 426  ☑ 426 
Q 473-973 T -500 ☑ -500
Q 32-989  T -957 ☒ -946
Q 323+2   T 325  ☑ 325 
Q 686-41  T 645  ☑ 645 
Q 771-65  T 706  ☒ 705 
Q 763-0   T 763  ☑ 763 
Q 366-3   T 363  ☑ 363 
Q 818+45  T 863  ☑ 863 
Q 64+811  T 875  ☑ 875 
Q 3-135   T -132 ☑ -132
Q 959+437 T 1396 ☒ 1406
Q 810+263 T 1073 ☑ 1073
Q 941-89  T 852  ☒ 842 
Q 18+515  T 533  ☑ 533 
Q 30-265  T -235 ☑ -235
Q 876-88  T 788  ☒ 799 
Q 998-3   T 995  ☑ 995 
Q 87+825  T 912  ☒ 922 
Q 296-4   T 292  ☑ 292 
Q 560-81  T 479  ☑ 479 
Q 960+6   T 966  ☑ 966 
Q 246-974 T -728 ☒ -733
Q 587-839 T -252 ☒ -242
Q 70+785  T 855  ☑ 855 
Q 659+3   T 662  ☑ 662 
Q 244+51  T 295  ☑ 295 
Q 193+69  T 262  ☒ 252 
Q 101+9   T 110  ☑ 110 
Q 830-24  T 806  ☒ 896 
Q 9-941   T -932